# Interfejs `Parcelable`

W tym przykładzie będziemy posiadać dwie aktywności, poprzez mechanizm `Intent` włączymy drugą aktywność oraz dzięki zaimplementowaniu interfejsu `Parcelable` oraz `Serializable` przekażemy cały obiekt. Do `activity_main.xml` dodajemy jeden `Button` przez naciśnięcie którego otworzymy drugą aktywność.

`Parcelable` i `Serializable` służą do zrobienia dokładnie tego samego; więc czym się różnią?
- `Parcelable` jest szybszy od `Serializable`
- `Serializable` jest interfejsem znacznikowym, więc implementacja jest szybsza
- `Serializable` tworzy wiele tymczasowych obiektów - jest niewydajny
- możemy przekazać całą tablicę obiektów za pomocą `Parcelable`
- w kotlinie mamy do dyspozycji adnotację `@Parcelize` dzięki której dostajemy domyślną implementację wszystkich metod interfejsu `Parcelable`

Interfejs `Serializable` jest jednym z najprostszych sposobów przenoszenia danych między różnymi częściami aplikacji w Androidzie. Służy do oznaczenia klasy, która może być **zserializowana**, czyli zamieniona na postać, która może być przechowywana lub przesyłana jako ciąg bajtów.

Kiedy klasa implementuje interfejs `Serializable`, można jej obiekty przekazać do innych komponentów aplikacji lub zapisać je na dysku w postaci pliku lub w bazie danych. Przy użyciu `Serializable` można serializować całe drzewa obiektów, w tym ich pola, podklasy itp.

Interfejs `Parcelable` jest alternatywnym sposobem przekazywania obiektów między komponentami w Androidzie. Podobnie jak `Serializable`, służy do zamiany obiektów na postać, która może być przechowywana lub przesyłana między procesami w systemie Android. Jednakże, w porównaniu z `Serializable`, `Parcelable` oferuje lepszą wydajność, szczególnie w przypadku przesyłania dużych obiektów między komponentami.

W przeciwieństwie do `Serializable`, `Parcelable` wymaga ręcznego zaimplementowania metod `writeToParcel()` i `createFromParcel()`, które służą do zamiany obiektu na postać Parcel i odczytanie z niego. `Parcel` to specjalny obiekt, który przechowuje binarną reprezentację danych i jest wykorzystywany do przesyłania danych między komponentami.

Implementacja interfejsu `Parcelable` może być bardziej pracochłonna niż `Serializable`, ale oferuje wiele zalet w kontekście Androida, szczególnie w przypadku przesyłania dużych obiektów między komponentami. Dzięki zastosowaniu `Parcelable` można osiągnąć lepszą wydajność i uniknąć problemów związanych z `Serializable`, takich jak problem z zapętlaniem w przypadku obiektów z referencjami cyklicznymi.

## `Parcelable`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".MainActivity">

    <Button
        android:id="@+id/sendButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_margin="16dp"
        android:text="SEND"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Następnie tworzymy drugą aktywność `SecondActivity.java` i uzupełniamy jej layout w pliku `second_activity.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".SecondActivity">

    <TextView
        android:id="@+id/textView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Przejdźmy do manifestu i określmy hierarchię aktywności

In [ ]:
        <activity
            android:name=".SecondActivity"
            android:parentActivityName=".MainActivity"
            android:exported="false" />

Następnie utwórzmy klasę której instancję przekażemy pomiędzy aktywnościami.

In [ ]:
public class Properties {
    private int a;
    private int b;
    private String c;

    public Properties(int a, int b, String c) {
        this.a = a;
        this.b = b;
        this.c = c;
    }

    public int getA() {
        return a;
    }

    public void setA(int a) {
        this.a = a;
    }

    public int getB() {
        return b;
    }

    public void setB(int b) {
        this.b = b;
    }

    public String getC() {
        return c;
    }

    public void setC(String c) {
        this.c = c;
    }
}

Aby być w stanie intancję takiej klasy przekazać do innej aktywności, musi ona implementować interfejs `Parcelable` (lub `Serializable` - więcej o różnicach poniżej).

In [ ]:
public class Properties implements Parcelable {
    private int a;
    private int b;
    private String c;

    public Properties(int a, int b, String c) {
        this.a = a;
        this.b = b;
        this.c = c;
    }

    protected Properties(Parcel in) {
        a = in.readInt();
        b = in.readInt();
        c = in.readString();
    }

    public static final Creator<Properties> CREATOR = new Creator<Properties>() {
        @Override
        public Properties createFromParcel(Parcel in) {
            return new Properties(in);
        }

        @Override
        public Properties[] newArray(int size) {
            return new Properties[size];
        }
    };

    public int getA() {
        return a;
    }

    public void setA(int a) {
        this.a = a;
    }

    public int getB() {
        return b;
    }

    public void setB(int b) {
        this.b = b;
    }

    public String getC() {
        return c;
    }

    public void setC(String c) {
        this.c = c;
    }

    @Override
    public int describeContents() {
        return hashCode();
    }

    @Override
    public void writeToParcel(Parcel parcel, int i) {
        parcel.writeInt(a);
        parcel.writeInt(b);
        parcel.writeString(c);
    }
}

Jakc widzimy musimy zaimplementować dwie metody, konstruktor oraz **companion object** `Creator`. Ponieważ obiekt towarzyszący jest sam w sobie jest obiektem więc nie ma żadnych ograniczeń - może rozszerzać klasy i implementować interfejsy. Jeżeli chcemy przekazać instancję klasy przez mechanizm `Intent` lub zachować w obiekcie `Bundle` w pierwszym kroku niejawnie zostanie wywołana metoda `writeToParcel`. Metoda ta opakowuje wszystkie właściwości klasy do obiektu `Parcel` wywołując metodę `writeToParcel` z odpowiednim typem. Przy odtwarzaniu instancji klasy `Properties` w pierwszym kroku wywoływana jest metoda `createFromParcel` znajdująca się w obiekcie towarzyszącym `Creator`. Metoda ta wywołuje drugorzędny konstruktor przyjmujący jako argument `Parcel`. Istotnym elementem tej implementacji jest konieczność zachowania kolejności przy przekazywaniu właściwości do i z `Parcel`. Przyjrzyjmy się konstruktorowi drugorzędnemu i metodzie `writeToParcel`.

Innymi słowy metoda `writeToParcel()` zapisuje pola obiektu do obiektu Parcel, a metoda `createFromParcel()` odczytuje te pola z `Parcel`. Następnie, w `Companion object`, definiujemy obiekt `CREATOR` typu `Parcelable.Creator<Properties>`, który jest odpowiedzialny za tworzenie instancji `Properies` z `Parcel`.

In [ ]:
protected Properties(Parcel in) {
    a = in.readInt();
    b = in.readInt();
    c = in.readString();
}

@Override
public void writeToParcel(Parcel parcel, int i) {
    parcel.writeInt(a);
    parcel.writeInt(b);
    parcel.writeString(c);
}

Pola `a`, `b` i `c` są zapisywane do `Parcel` i odczytywane w konstruktorze dokładnie w tej samej kolejności - jest to niezbędne do poprawnego działania - zweróćmy uwagę że nie posługujemy się tutaj żadnym kluczem, czy jakimś unikalnym identyfikartorem.

Ostatnią metodą jest `describeContent` - w interfejsie zdefiniowana jest stała `CONTENT_FILE_DESCRIPTION` która powinna zostać użyta w tej metodzie do zwrócenia maski bitowej - tutaj zwrócimy wynik wwywołania metody `hashCode`.

In [ ]:
@Override
public int describeContents() {
    return hashCode();
}

Przejdźmy do implementacji metody `onClick` przycisku na głównej aktyności. Tym razem zrobimy to nieco inaczej, przechodzimy do pliku `MainActivity.java`. Do metody `onCreate` dodajemy

In [ ]:
Button button = findViewById(R.id.sendButton);
button.setOnClickListener(new View.OnClickListener() {
    @Override
    public void onClick(View view) {

    }
});

Ustawiamy `onClickListener` przycisku i jako argument tworzymy **klasę anonimową** o typie  interfejsu `OnClickListener` znajdującego się w klasie `View`. Klasa `View` jest główną klasą wszystkich elementów interfejsu użytkownika - jest rozszerzana przez wszystkie klasy. Częściej spotykaną jest implementacja za pomocą wyrażenia **lambda**

In [ ]:
binding.sendButton.setOnClickListener(view -> {

});

Dodajmy implementację `onClick`

In [ ]:
properties = new Properties(1, 2, "String");
serializableProperties = new SerializableProperties(11, 12, "Serializable");

binding.sendButton.setOnClickListener(view -> {
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_KEY, properties);
    startActivity(intent);
});

Tworzymy klucz dla `Intent`, następnie tworzymy instancję `Properties` oraz sam `Intent`. Wskazujemy kontekst jako `this` oraz cel którym jest `SecoondActivity.class`. Następnie dodajemy za pomocą metody `putExtra` nasz obiekt `prop` i wywołujemy `startActivity` w celu włączenia drugiej aktywności.

W następnym kroku odbierzemy dane w `SecondActivity` i rozpakujemy obiekt `Properties`. Przejdźmy do pliku `SecondActivity.java`

In [ ]:
package com.example.a42_parcelableserializablebasics_java;

import androidx.appcompat.app.AppCompatActivity;

import android.content.Intent;
import android.os.Build;
import android.os.Bundle;

import com.example.a42_parcelableserializablebasics_java.databinding.ActivitySecondBinding;

public class SecondActivity extends AppCompatActivity {

    private ActivitySecondBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivitySecondBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        Intent intent = getIntent();
        Properties properties;
        if (android.os.Build.VERSION.SDK_INT >= Build.VERSION_CODES.TIRAMISU) {
            properties = getIntent().getParcelableExtra(MainActivity.EXTRA_KEY, Properties.class);
        } else {
            properties = getIntent().getParcelableExtra(MainActivity.EXTRA_KEY);
        }

        String result = "";

        if(properties != null)
            result = properties.getA() + properties.getB() + " " + properties.getC();

        binding.textView.setText(result);
    }
}

Tworzymy `Intent` i wywoołujemy metodę `getIntent`, następnie tworzymy obiekt typu `Properties` - tutaj niejawnie zoostanie wywołany konstruktor z argumentem typu `Parcel`. Wyciągamy obiekt za pomocą metody `getParcelableExtra` jako argument podając klucz. Do odpowiednich pól możemy dostać się poprzez odpowiednie **gettery**. Możemy przetestować aplikację.

## **Interfejs `Serializable`**

Dodajmy do projektu kolejną klasę `SerializableProperties`, która implementuje interfejs `Serializable` i powtórzmy wszystkie czynności.

In [ ]:
package pl.edu.uwr.pum.parcelableexamplejava;

import java.io.Serializable;

public class SerializableProperties implements Serializable {
        private int a;
        private int b;
        private String c;

        public SerializableProperties(int a, int b, String c) {
            this.a = a;
            this.b = b;
            this.c = c;
        }

        public int getA() {
            return a;
        }

        public void setA(int a) {
            this.a = a;
        }

        public int getB() {
            return b;
        }

        public void setB(int b) {
            this.b = b;
        }

        public String getC() {
            return c;
        }

        public void setC(String c) {
            this.c = c;
        }
}

Interfejs `Serializable` jest **interfejsem znacznikowym** więc nie jest koonieczne implementowanie żadnych metod

W klasie `MainActivity.java` dodajmy jeszcze jedno pole

In [ ]:
public static String EXTRA_SERIALIZABLE = "serializable_example";

Następnie utworzymy instancję klasy `SerializableProperties`

In [ ]:
serProp = new SerializableProperties(11, 12, "Serializable");

i dodamy ją do wcześniej utworzonego `Intent`

In [ ]:
binding.sendButton.setOnClickListener(view -> {
    Intent intent = new Intent(this, SecondActivity.class);
    intent.putExtra(EXTRA_KEY, properties);
    intent.putExtra(EXTRA_SERIALIZABLE, serializableProperties);
    startActivity(intent);
});

Przejdźmy do `SecondActivity`, rozpakujmy nasz drugi obiekt i dodajmy jego pola do `TextView`.

In [ ]:
package com.example.a42_parcelableserializablebasics_java;

import androidx.appcompat.app.AppCompatActivity;

import android.content.Intent;
import android.os.Build;
import android.os.Bundle;

import com.example.a42_parcelableserializablebasics_java.databinding.ActivitySecondBinding;

public class SecondActivity extends AppCompatActivity {

    private ActivitySecondBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivitySecondBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());

        Intent intent = getIntent();
        Properties properties;
        if (android.os.Build.VERSION.SDK_INT >= Build.VERSION_CODES.TIRAMISU) {
            properties = getIntent().getParcelableExtra(MainActivity.EXTRA_KEY, Properties.class);
        } else {
            properties = getIntent().getParcelableExtra(MainActivity.EXTRA_KEY);
        }

        SerializableProperties serializableProperties;
        if (android.os.Build.VERSION.SDK_INT >= Build.VERSION_CODES.TIRAMISU) {
            serializableProperties = getIntent().getSerializableExtra(
                MainActivity.EXTRA_SERIALIZABLE, 
                SerializableProperties.class);
        } else {
            serializableProperties = (SerializableProperties) getIntent()
                .getSerializableExtra(MainActivity.EXTRA_SERIALIZABLE);
        }

        String result = "";

        if(properties != null && serializableProperties != null)
            result = properties.getA() + 
                     properties.getB() + " " + 
                     properties.getC() + "---" + 
                     serializableProperties.getA() + "\n" +
                     serializableProperties.getB() + " " + 
                     serializableProperties.getC();

        binding.textView.setText(result);
    }
}

Zwróćmy uwagę że przy wywołaniu metody `getSerializableExtra` dostajjjemy obiekt o typie interfejsu `Serializable` i potrszebujemy obiekt typu `SerializableProperties`. Ponieważ nasza klasa implementuje ten interfejs, możemy bezpiecznie rzutować na `SerializableProperties`.

Po uruchomieniu aplikacji powinniśmy dostać podobny wynik jak poprzednio. Widzimy że właściwie mogę zrobić dokładnie to samo przy pomocy dwóch interfejsów - `Parcelable` i `Serializable`; więc czym się różnią?
- `Parcelable` jest szybszy od `Serializable`
- `Serializable` jest interfejsem znacznikowym, więc implementacja jest szybsza
- `Serializable` tworzy wiele tymczasowych obiektów - jest niewydajny
- możemy przekazać całą tablicę obiektów za pomocą `Parcelable`
- w kotlinie mamy do dyspozycji adnotację `@Parcelize` dzięki której dostajemy domyślną implementację wszystkich metod interfejsu `Parcelable`